# Unit 1 Special Content: Optuna Guide

In this notebook, we shall see how to use Optuna to perform hyperparameter tuning of Unit 1's <a href="https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html" target="_blank">`PPO`</a> model (created using Stable-Baselines3 for the `"LunarLander-v2"` Gym environment).

Optuna is an open-source, automatic hyperparameter optimization framework. You can read more about it <a href="https://tech.preferred.jp/en/blog/optuna-release/" target="_blank">here</a>.

**Prerequisite:** Before going through this notebook, you should have completed the <a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/unit1/unit1.ipynb" target="_blank">Unit 1 hands-on</a>.

## Virtual Display Setup

We'll need to generate a replay video. To do so in Colab, we need to have a virtual display to be able to render the environment (and thus record the frames).

The following cell will install virtual display libraries.

In [ ]:
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay

Now, let's create & start a virtual display.

In [ ]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Dependencies, Imports & Gym Environments

Let's install all the other dependencies we'll need.

In [ ]:
!pip install gym[box2d]
!pip install stable-baselines3[extra]
!pip install pyglet
!pip install ale-py==0.7.4 # To overcome an issue with Gym (https://github.com/DLR-RM/stable-baselines3/issues/875)
!pip install optuna
!pip install huggingface_sb3

Next, let's perform all the necessary imports.

In [ ]:
import gym

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv

import optuna
from optuna.samplers import TPESampler

from huggingface_hub import notebook_login
from huggingface_sb3 import package_to_hub

Finally, let's create our Gym environments. The training environment is a vectorized environment:

In [ ]:
env = make_vec_env("LunarLander-v2", n_envs=16)
env

And the evaluation environment is a separate environment:

In [ ]:
eval_env = Monitor(gym.make("LunarLander-v2"))

We are now ready to dive into hyperparameter tuning!

## Hyperparameter Tuning

First, let's define a `run_training()` function that trains a single model (using a particular combination of hyperparameter values), and returns a score. 

The score tells us how good the particular combination of hyperparameters is. (In our case, the score is `mean_reward - std_reward`, which is being used in the <a href="https://huggingface.co/spaces/chrisjay/Deep-Reinforcement-Learning-Leaderboard" target="_blank">leaderboard</a>.) 

The function takes a very special argument - `params`, which is a dictionary. **The keys of this dictionary are the names of the hyperparameters we're tuning**, and **the values are sampled at each trial by Optuna's sampler** (from ranges that we'll specify soon).

For example, in a particular trial, `params` might look like this:

```
{'n_epochs': 5, 'gamma': 0.9926, 'total_timesteps': 559_621}
```

And in another trial, `params` might look like this:

```
{'n_epochs': 3, 'gamma': 0.9974, 'total_timesteps': 1_728_482}
```

In [ ]:
def run_training(params, verbose=0, save_model=False):
    model = PPO(
        policy='MlpPolicy', 
        env=env, 
        n_steps=1024,
        batch_size=64, 
        n_epochs=params['n_epochs'], # We're tuning this.
        gamma=params['gamma'], # We're tuning this.
        gae_lambda=0.98, 
        ent_coef=0.01, 
        verbose=verbose
    )
    model.learn(total_timesteps=params['total_timesteps']) # We're tuning this.

    mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=50, deterministic=True)
    score = mean_reward - std_reward

    if save_model:
        model.save("PPO-LunarLander-v2")

    return model, score

Next, we define another function - `objective()`. This function has a single parameter `trial`, which is an object of type `optuna.trial.Trial`. Using this `trial` object, we specify the ranges for the different hyperparameters we want to explore:

- For `n_epochs`: We want to explore integer values between `3` and `5`.
- For `gamma`: We want to explore floating point values between `0.9900` and `0.9999` (drawn from a uniform distribution).
- For `total_timesteps`: We want to explore integer values between `500_000` and `2_000_000`.

**Note:** If you have more time available, then you can tune other hyperparameters too. Moreover, you can explore wider ranges for each hyperparameter.

The `trial.suggest_int()` and `trial.suggest_uniform()` methods are used by Optuna to suggest hyperparameter values in the ranges specified. The suggested combination of values are then used to train a model and return the score.

In [ ]:
def objective(trial):
  params = {
      "n_epochs": trial.suggest_int("n_epochs", 3, 5), 
      "gamma": trial.suggest_uniform("gamma", 0.9900, 0.9999), 
      "total_timesteps": trial.suggest_int("total_timesteps", 500_000, 2_000_000)
  }
  model, score = run_training(params)
  return score

Finally, we use Optuna's `create_study()` function to create a study, passing in:

- `sampler=TPESampler()`: This specifies that we want to employ a Bayesian optimization algorithm called Tree-structured Parzen Estimator. Other options are `GridSampler()`, `RandomSampler()`, etc. (The full list can be found <a href="https://optuna.readthedocs.io/en/stable/reference/samplers.html" target="_blank">here</a>.)
- `study_name="PPO-LunarLander-v2"`: This is a name we give to the study (optional).
- `direction="maximize"`: This is to specify that our objective is to maximize (not minimize) the score.

Once our study is created, we call the `optimize()` method on it, specifying that we want to conduct `10` trials.

**Note:** If you have more time available, then you can conduct more than `10` trials.

**Warning:** The below code cell will take quite a bit of time to run!

In [ ]:
study = optuna.create_study(sampler=TPESampler(), study_name="PPO-LunarLander-v2", direction="maximize")
study.optimize(objective, n_trials=10)

Now that all the `10` trials have concluded, let's print out the score and hyperparameters of the best trial.

In [ ]:
print("Best trial score:", study.best_trial.values)
print("Best trial hyperparameters:", study.best_trial.params)

## Recreating & Saving The Best Model

Let's recreate the best model and save it.

In [ ]:
model, score = run_training(study.best_trial.params, verbose=1, save_model=True)

## Pushing to Hugging Face Hub

To be able to share your model with the community, there are three more steps to follow:

1. (If not done already) create a Hugging Face account -> https://huggingface.co/join

2. Sign in and then, get your authentication token from the Hugging Face website.

- Create a new token (https://huggingface.co/settings/tokens) **with write role**.
- Copy the token.
- Run the cell below and paste the token.

In [ ]:
notebook_login()
!git config --global credential.helper store

If you aren't using Google Colab or Jupyter Notebook, you need to use this command instead: `huggingface-cli login`

3. We're now ready to push our trained agent to the Hub using the `package_to_hub()` function.

Let's fill in the arguments of the `package_to_hub` function:

- `model`: our trained model

- `model_name`: the name of the trained model that we defined in `model.save()`

- `model_architecture`: the model architecture we used (in our case `"PPO"`)

- `env_id`: the name of the environment (in our case `"LunarLander-v2"`)

- `eval_env`: the evaluation environment

- `repo_id`: the name of the Hugging Face Hub repository that will be created/updated `(repo_id="{username}/{repo_name}")` (**Note:** A good `repo_id` is `"{username}/{model_architecture}-{env_id}"`.)

- `commit_message`: the commit message

In [ ]:
model_name = "PPO-LunarLander-v2"
model_architecture = "PPO"
env_id = "LunarLander-v2"
eval_env = DummyVecEnv([lambda: gym.make(env_id)])
repo_id = "Sadhaklal/PPO-LunarLander-v2"
commit_message = "Upload best PPO LunarLander-v2 agent (tuned with Optuna)."

The following function call will evaluate the agent, record a replay, generate a model card, and push your agent to the Hub.

In [ ]:
package_to_hub(
    model=model, 
    model_name=model_name, 
    model_architecture=model_architecture, 
    env_id=env_id, 
    eval_env=eval_env, 
    repo_id=repo_id, 
    commit_message=commit_message
)

That's it! You now know how to perform hyperparameter tuning of Stable-Baselines3 models using Optuna.

To get even better results, try tuning the other hyperparameters of your model.

## Final Tips

1. Read the <a href="https://optuna.readthedocs.io/en/stable/index.html" target="_blank">Optuna documentation</a> to get more familiar with the library and its features.
2. You may have noticed that hyperparameter tuning is a time consuming process. However, it can be sped up significantly using parallelization. Check out <a href="https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/004_distributed.html" target="_blank">this guide</a> on how to do so.